In [0]:
# Databricks notebook: Agregaciones y métricas Gold para OpenSky

from pyspark.sql.functions import avg, count, col, round, desc

# 1. Cargar datos desde Silver
df_silver = spark.table("default.silver_opensky")

# ========================================================
# 2. Tabla: Altitud promedio por país
# ========================================================
df_altitude = df_silver.groupBy("origin_country") \
    .agg(round(avg("geo_altitude"), 2).alias("avg_geo_altitude")) \
    .orderBy(desc("avg_geo_altitude"))

df_altitude.write.format("delta").mode("overwrite").saveAsTable("default.gold_avg_altitude")
print("✅ Tabla creada: default.gold_avg_altitude")

# ========================================================
# 3. Tabla: Velocidad promedio por país
# ========================================================
df_velocity = df_silver.groupBy("origin_country") \
    .agg(round(avg("velocity"), 2).alias("avg_velocity")) \
    .orderBy(desc("avg_velocity"))

df_velocity.write.format("delta").mode("overwrite").saveAsTable("default.gold_velocity_by_country")
print("✅ Tabla creada: default.gold_velocity_by_country")

# ========================================================
# 4. Tabla: Número de vuelos en el aire vs en tierra por país
# ========================================================
df_airborne = df_silver.groupBy("origin_country", "on_ground") \
    .count() \
    .orderBy(desc("count"))

df_airborne.write.format("delta").mode("overwrite").saveAsTable("default.gold_airborne_distribution")
print("✅ Tabla creada: default.gold_airborne_distribution")

# ========================================================
# 5. Tabla: Número total de vuelos por país
# ========================================================
df_total_flights = df_silver.groupBy("origin_country") \
    .count() \
    .orderBy(desc("count"))

df_total_flights.write.format("delta").mode("overwrite").saveAsTable("default.gold_total_flights")
print("✅ Tabla creada: default.gold_total_flights")

# ========================================================
# 6. Tabla: Top 10 altitudes individuales más altas
# ========================================================
df_top_altitudes = df_silver.select("callsign", "origin_country", "geo_altitude") \
    .where(col("geo_altitude").isNotNull()) \
    .orderBy(desc("geo_altitude")) \
    .limit(10)

df_top_altitudes.write.format("delta").mode("overwrite").saveAsTable("default.gold_top_altitudes")
print("✅ Tabla creada: default.gold_top_altitudes")

# ========================================================
# 7. Tabla: Promedio por país de vuelos que están en el aire
# ========================================================
df_avg_airborne = df_silver.filter("on_ground = false") \
    .groupBy("origin_country") \
    .agg(round(avg("geo_altitude"), 2).alias("avg_altitude_airborne"),
         round(avg("velocity"), 2).alias("avg_velocity_airborne")) \
    .orderBy(desc("avg_altitude_airborne"))

df_avg_airborne.write.format("delta").mode("overwrite").saveAsTable("default.gold_avg_airborne_stats")
print("✅ Tabla creada: default.gold_avg_airborne_stats")
